In [107]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 

# Example 4. PMF (Passenger Mix Flow) model

In [108]:
# Define the inputs

# L: set of flights
flights = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Flight').set_index('Flight Number')

# P: set of passenger itineraries
paths = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Itinerary').set_index('Itin No.')
paths['Leg 2'] = paths['Leg 2'].astype('Int64')

# P_p: set of passenger itineraries with recapture rate from itinerary p
P_p = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Recapture Rate').set_index(['p','r'])

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths = paths.to_dict(orient='index')
flights = flights.to_dict(orient='index')
P_p = P_p.to_dict(orient='index')

path_list = list(paths.keys())
flight_list = list(flights.keys())

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if type(paths[key]['Leg 1']) == int and type(paths[key]['Leg 2']) == int:
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif type(paths[key]['Leg 1']) == int:
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']


In [109]:
# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

In [110]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flight_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flight_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

# ds_i demand spill for flight i = Q_i - capacity of flight i
ds_i = {}
for i in flight_list:
    ds_i[i] = Q_i[i] - flights[i]['Capacity']

In [111]:
# Add entries to P_p for path 0 with a recapture rate of 1
for p in paths:
    P_p[p,999] = {'Recapture rate': 1}
    P_p[999,p] = {'Recapture rate': 0}

path_list = list(paths.keys())
flight_list = list(flights.keys())

In [112]:
initial = [999]

In [113]:
# Implementation using column generation algorithm (CGA)

# Define the restricted master problem (RMP)
# All the spillage is reallocated to path 999

# Define the model
m = gp.Model('PMF')

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}
for p in path_list:
    for r in initial:
        t[p,r] = m.addVar(name='t_'+str(p)+'_'+str(r))

In [114]:
# Define the objective function
# MINIMIZE (double sum of fare_p - bpr * fare_r) * t_pr
of = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
m.setObjective(of, GRB.MINIMIZE)

# Define the constraints
# Constraint 1: sum sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(s_ip[i,p] * t[p,r] for p in path_list) - 
              gp.quicksum(s_ip[i,p] * P_p[(r,p)]['Recapture rate'] * t[p,r] for p in path_list for r in initial) >= 
              ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
m.addConstrs((t[p,r] <= paths[p]['Demand'] for p in path_list), name='σ')

# Constraint 3: sum t_pr >= 0 for all p
m.addConstrs((t[p,r] >= 0 for p in path_list), name='c3')

m.update()

In [115]:
# Solve the model
m.optimize()


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 16 columns and 52 nonzeros
Model fingerprint: 0x71139809
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+02]
Presolve removed 39 rows and 11 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8055000e+04   1.162500e+01   0.000000e+00      0s
       2    4.6580000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.658000000e+04


In [116]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %g' % m.objVal)
print('Optimal solution:')
for v in m.getVars():
    if v.x > 0:
        print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':
        # get only the flight number from the constraint name
        flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
        sigma_dual[path_num_sigma] = c.Pi
        

Optimal objective value: 46580
Optimal solution:
t_2_999 = 12
t_5_999 = 50
t_6_999 = 12
t_7_999 = 62
t_8_999 = 70
t_9_999 = 32
t_10_999 = 87
t_12_999 = 72
t_13_999 = 30
Dual variables:
π[102] = 100
π[104] = 170
π[202] = 80
π[302] = 140
π[203] = 180
π[101] = 120
σ[5] = -90
σ[8] = -250
σ[13] = -100


In [117]:
# Create the pricing problem (PP)

tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in P_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual[i] for i in paths[p]['Legs'])) -
                      (P_p[(p,r)]['Recapture rate']) *
                      (paths[r]['Fare'] - sum(pi_dual[j] for j in paths[r]['Legs'])) -
                      (sigma_dual[p]))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr

new_pairs = list(tpr_prime.keys())

# Add the new columns to the model
new_p = []
for p,r in new_pairs:
    new_p.append(p)

# Current variables
current_vars = []
for v in m.getVars():
    current_vars.append(v.varName)

path_list.extend(new_p)

In [118]:
# Add the new columns to the RMP
for i, (p,r) in enumerate(new_pairs):
    t_pr[len(current_vars)+i] = m.addVar(name='t_' + str(p) +'_'+ str(r))
m.update()

# Update the objective function
of += gp.quicksum((paths[p]['Fare'] - P_p[(p, r)]['Recapture rate'] * paths[r]['Fare']) * t_pr[p] for p,r in new_pairs)
m.setObjective(of, GRB.MINIMIZE)

# Update the constraints
m.addConstrs((gp.quicksum(s_ip[i, p] * t_pr[p] for p in path_list) -
              gp.quicksum(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t_pr[len(current_vars)+u] for u, (p,r) in enumerate(new_pairs)) >=
               ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
m.addConstrs((t_pr[p] <= paths[p]['Demand'] for p in path_list), name='σ')

# Constraint 3: sum t_pr >= 0 for all p
m.addConstrs((t_pr[p] >= 0 for p in path_list), name='c3')

# Update the model
m.update()


GurobiError: Variable not in model

In [103]:
for i in flight_list:
    for u, p in enumerate(path_list):
        print(u,p)
        print(i, s_ip[i, p], t_pr[0])


0 1


KeyError: 0

In [79]:
# for i in flight_list:
#     for u, (p,r) in enumerate(new_pairs):
#         print(u,p,r)
#         print(i,"|", p,"|", r,"|", s_ip[i, r],"|", P_p[(p, r)]['Recapture rate'] , t_pr[len(current_vars)+u],"|", (ds_i[i]))

In [81]:
# Solve the model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 90 rows, 19 columns and 113 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [5e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0320000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.032000000e+04


In [82]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
#print('Optimal objective value: %g' % m.objVal)
print('Optimal solution:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':
        pi_dual[c.ConstrName] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        sigma_dual[c.ConstrName] = c.Pi
        

Optimal solution:
t_p999[1] = 0
t_p999[2] = 12
t_p999[3] = 0
t_p999[4] = 0
t_p999[5] = 50
t_p999[6] = 12
t_p999[7] = 62
t_p999[8] = 70
t_p999[9] = 32
t_p999[10] = 87
t_p999[11] = 0
t_p999[12] = 72
t_p999[13] = 30
t_p999[14] = 0
t_p999[15] = 0
t_p999[999] = 0
t_2_1 = 0
t_3_4 = 0
t_12_11 = 0
Dual variables:
π[102] = 100
π[104] = 325
π[202] = 80
π[302] = 140
π[203] = 345
π[101] = 120
σ[5] = -90
σ[8] = -570
σ[13] = -100


In [83]:
pi_dual

{'π[102]': 0.0,
 'π[301]': 0.0,
 'π[201]': 0.0,
 'π[104]': 0.0,
 'π[202]': 0.0,
 'π[302]': 0.0,
 'π[303]': 0.0,
 'π[204]': 0.0,
 'π[203]': 0.0,
 'π[101]': 0.0}